Attempts to solve https://www.janestreet.com/puzzles/current-puzzle/
Differences are:
1) We use nine of the ten digits (0-9) in completing this grid instead of (1-9)
2) We're trying to find the sudoku board with the highest possible greatest-common-divisor over the 9-digit number formed by the rows of the grid

In [15]:
from z3 import *
from pprint import pprint

# Create integer powers of 10
POW = [1e8, 1e7, 1e6, 1e5, 1e4, 1e3, 1e2, 1e1, 1e0]
POW = [int(i) for i in POW]

In [16]:
# Create an integer variable for each cell in the 9x9 sudoku grid
X = [ [ Int(f"x_%s_%s" % (i+1, j+1)) for j in range(9) ] 
      for i in range(9) ]

# Each cell must contain a digit (0 to 9)
cells_c = [And(0 <= X[i][j], X[i][j] <= 9) for i in range(9) for j in range(9)]

excluded_digit = Int("excluded")
# Excluded digit is between 0 and 9 inclusive
excluded_digit_c = [And(0 <= excluded_digit, excluded_digit <= 9)]

# Each cell must contain a digit (0 to 9)
cells_c = [And(0 <= X[i][j], X[i][j] <= 9) for i in range(9) for j in range(9)]

# Every digit has to be placed exactly once in each row (with the exception of the excluded digit)
rows_c = [Distinct(X[i] + [excluded_digit]) for i in range(9)]

# Every digit has to be placed exactly once in each column (with the exception of the excluded digit)
cols_c = [Distinct([X[i][j] for i in range(9)] + [excluded_digit]) for j in range(9)]

# Every digit has to be placed exactly once in each 3x3 subgrid (with the exception of the excluded digit)
sq_c = [
    Distinct(
        [X[3 * i0 + i][3 * j0 + j] for i in range(3) for j in range(3)]
        + [excluded_digit]
    )
    for i0 in range(3)
    for j0 in range(3)
]

def create_number(number_var, num_list):
    # TODO
    num = 0
    for ind, val in enumerate(num_list):
        num += val * POW[ind]
    return number_var == num

# Create the divisor variable
divisor = Int("divisor")
# Bit of demo magic to make sure we find the answer in the face of the unknown ;)
divisor_c = [divisor == 9]
numbers = [Int(f"num_{i}") for i in range(9)]
# Each constructed number is equal to the dot product of its row
numbers_dot_c = [create_number(numbers[ind], row) for ind, row in enumerate(X)]
# Each number is divisible by the divisor
numbers_div_c = [number % divisor == 0 for number in numbers]

In [17]:
# Chain all the conditions together
sudoku_c = (
    excluded_digit_c + cells_c + rows_c + cols_c + sq_c + numbers_dot_c + numbers_div_c + divisor_c
)

In [18]:
instance = (
    (-1, -1, -1, -1, -1, -1, -1, 2, -1),
    (-1, -1, -1, -1, -1, -1, -1, -1, 5),
    (-1, 2, -1, -1, -1, -1, -1, -1, -1),
    (-1, -1, 0, -1, -1, -1, -1, -1, -1),
    (-1, -1, -1, -1, -1, -1, -1, -1, -1),
    (-1, -1, -1, 2, -1, -1, -1, -1, -1),
    (-1, -1, -1, -1, 0, -1, -1, -1, -1),
    (-1, -1, -1, -1, -1, 2, -1, -1, -1),
    (-1, -1, -1, -1, -1, -1, 5, -1, -1),
)

instance_c = [
    If(instance[i][j] == -1, True, X[i][j] == instance[i][j])
    for i in range(9)
    for j in range(9)
]

s = Solver()
# s = Optimize()
s.add(sudoku_c + instance_c)
# s.maximize(divisor)

if s.check() == sat:
    m = s.model()
    r = [[m.evaluate(X[i][j]) for j in range(9)] for i in range(9)]
    print_matrix(r)
    for n in numbers:
        print(m.evaluate(n))
    print(f"excluded_digit={m.evaluate(excluded_digit)}")
    print(f"divisor={m.evaluate(divisor)}")
else:
    print("failed to solve")

[[1, 3, 6, 4, 5, 7, 0, 2, 8],
 [0, 7, 8, 6, 2, 1, 4, 3, 5],
 [4, 2, 5, 0, 3, 8, 1, 6, 7],
 [2, 4, 0, 5, 8, 6, 3, 7, 1],
 [8, 6, 7, 3, 1, 4, 2, 5, 0],
 [3, 5, 1, 2, 7, 0, 6, 8, 4],
 [6, 1, 2, 7, 0, 5, 8, 4, 3],
 [5, 8, 3, 1, 4, 2, 7, 0, 6],
 [7, 0, 4, 8, 6, 3, 5, 1, 2]]
136457028
78621435
425038167
240586371
867314250
351270684
612705843
583142706
704863512
excluded_digit=9
divisor=9
